In [14]:
import pandas as pd, os

# add digittal project with abstract filter_1
from source.c_is_digital_project import add_dp
# Get project description, project componets
from source.d_project_desc_components import get_context,  get_components
# Get lines_text with azure api
from source.e_azure_detect import AzureDocument
# Clasification digital projects filter_2
from source.g_pillars import classification, is_digital_filter2, valid_tokens
from source.g_pillars import add_categories_cols

## First Filter

In [ ]:
data_original = "data/csv/01_merged_data_with_powerBI.csv"
data = pd.read_csv(data_original)

list_comparing = data.sample(5, random_state=999)
eval = list_comparing.merge(data, how='left')

In [35]:

print("Ingrese el path del archivo, debe contenter las siguientes columnas ['projectid', 'txturl', 'pdfurl']")
print("")

digital_comparing = "./00_test/test.csv"
if os.path.exists(digital_comparing):
	with_digital = pd.read_csv(digital_comparing)
else: 
	with_digital = add_dp(eval)
	with_digital.to_csv(digital_comparing)
# eval

Ingrese el path del archivo, debe contenter las siguientes columnas ['projectid', 'txturl', 'pdfurl']



Fitler with gpt (1)

In [19]:
is_digital_project = with_digital.query('is_digital_project == 1')
no_digital_project = with_digital.query('is_digital_project != 1')

relevant columns

In [20]:
cols = ['projectid', 'txturl', 'pdfurl']
is_digital = is_digital_project[cols]
is_digital['project_description'] = is_digital['txturl'].apply(lambda x: get_context(x))
is_digital['project_components'] = is_digital['txturl'].apply(lambda x: get_components(x))
is_digital

,projectid,txturl,pdfurl,project_description,project_components
0,P161876,http://documents.worldbank.org/curated/en/4020...,http://documents.worldbank.org/curated/en/4020...,II. PROGRAM DESCRIPTION\nA. Government Program...,None
4,P148850,http://documents.worldbank.org/curated/en/8823...,http://documents.worldbank.org/curated/en/8823...,III. PROJECT DESCRIPTION\nA. Project Component...,A. Project Components\n24. The project will in...


2 attemp with azure 

In [21]:
print('ingrese el path del archivo pass_key que contentga el el "ENDPOINT", "API-KEY"')
azure = AzureDocument("./pass_keys.json")
def azure_text(row):
	if row['project_description'] is not None and row['project_components'] is not None:
		return 'None'
		# row['project_description']
	else:
		return azure.lines_data(row['pdfurl'])
	
is_digital['data_text_azure'] = is_digital.apply(azure_text, axis=1)
is_digital['project_description_azure'] = is_digital['data_text_azure'].apply(
	lambda x : get_context(x)
)
is_digital['project_components_azure'] = is_digital['data_text_azure'].apply(
	lambda x : get_components(x, url = None)
)


## Filter 2
run gpt, classify projects

In [24]:
is_digital['text_to_classification'] = is_digital.apply(valid_tokens, axis =1)

is_digital['classification'] = is_digital['text_to_classification'].apply(lambda x: classification(x))


In [27]:
# import json
# is_digital['classification'] = is_digital['classification'].apply(lambda x: json.load(x))
is_digital['is_dig_filter2'] = is_digital['classification'].apply(lambda x: is_digital_filter2(x))
is_digital = add_categories_cols(is_digital)


In [37]:
print('ingerse el path y nombre con la extencion .csv para guardar el resultado')
df = with_digital.merge(is_digital, how = 'left')
print(df)
df.to_csv('./00_test/result.csv')

ingerse el path y nombre con la extencion .csv para guardar el resultado
   Unnamed: 0 projectid  approval_fy        id  last_modified_date  \
0        1492   P161876         2018  29890848                 NaN   
1        1596   P152309         2016  25489468                 NaN   
2        1811   P163742         2019  30853638                 NaN   
3        1285   P147486         2015  19705540                 NaN   
4        1241   P148850         2015  24440102                 NaN   

                                        admreg     admreg_key  authors  \
0                                Africa,Africa  119222,119222      NaN   
1  East Asia and Pacific,East Asia and Pacific  119225,119225      NaN   
2                        South Asia,South Asia  119231,119231      NaN   
3                                Africa,Africa  119222,119222      NaN   
4                                Africa,Africa  119222,119222      NaN   

       count  count_key  ... Digital Public Platforms Categor